# Content-Based Filtering

### 1. 개요
- **사용자가 선호하는 아이템의 콘텐츠 정보**와 유사한 아이템을 추천하는 방식
    - 콘텐츠 정보 예시 : 장르, 키워드, 설명 등

### 2. 장단점
- 장점
    - 사용자에 대한 피드백 데이터가 적어도 협업 필터링 대비 추천 품질 좋음 (**relatively less** Cold Start Problem)
        - 아예 없을 경우, 회원가입 시 선호하는 장르 등에 대한 정보를 받을 수 있음
    - 새로운 아이템도 추천 바로 가능 (협업 필터링은 피드백 데이터 필요)
    - 설명 가능성이 높음 (Explainability)
- 단점
    - 기존 선호도와 유사한 아이템만 추천 (새로운 취향 발견 어려움, Filter Bubble)
    - 아이템의 정보가 충분해야 가능

### 3. 구현
1. Feature Extraction
    - 아이템의 주요 특징을 추출
    - 예시
        - 영화 : 장르, 감독, 출연 배우, 줄거리
        - 상품 : 카테고리, 가격, 브랜드
2. Embedding
    - 아이템의 특징을 벡터로 변환
    - 주요 방법
        - TF-IDF : 단어 빈도 기반
        - Word2Vec : 단어 의미 학습
        - BERT Embedding : 문맥 반영
            - Transformer 모델, 동음이의어 뜻 차이 반영 가능
3. Similarity Calculation
    - 유사도 계산
    - 주요 방법
        - Cosine Similarity
        - Euclidian Distance
4. Recommendation
    - 사용자가 선호하는 아이템과 유사도가 높은 아이템 추천
    - 예시
        - 센과 치히로 좋아하는 사용자에게 하울의 움직이는 성 추천
        - viva la vida 좋아하는 사용자에게 fix you 추천

### 4. 활용 사례
- Netflix 비슷한 장르 영화 추천
- Spotify 비슷한 장르 음악 추천
- Coupang 동일 브랜드 상품 추천

### 5. 관련 수식
#### 1. 아이템 벡터 표현
각 아이템 $ i $ 는 특성(feature) 벡터 $ \mathbf{x}_i $ 로 표현됨

$$
\mathbf{x}_i = (x_{i1}, x_{i2}, ..., x_{in})
$$

#### 2. 사용자 프로파일 벡터
사용자 $ u $ 는 선호하는 아이템들의 가중합으로 표현됨

$$
\mathbf{p}_u = \frac{\sum_{i \in I_u} w_{u,i} \mathbf{x}_i}{\sum_{i \in I_u} w_{u,i}}
$$

여기서
- $ I_u $ : 사용자 $ u $ 가 평가한 아이템 집합
- $ w_{u,i} $ : 사용자 $ u $ 가 아이템 $ i $ 에 부여한 가중치(예: 평점)
- $ \mathbf{x}_i $ : 아이템 $ i $ 의 특성 벡터

#### 3. 유사도 계산
사용자 프로파일 $ \mathbf{p}_u $ 와 아이템 $ \mathbf{x}_i $ 간의 유사도

$$
\text{sim}(\mathbf{p}_u, \mathbf{x}_i) = \frac{\mathbf{p}_u \cdot \mathbf{x}_i}{\|\mathbf{p}_u\| \|\mathbf{x}_i\|}
$$

확장하여, 개별 특성 차원에서 유사도를 나타내면

$$
\text{sim}(\mathbf{p}_u, \mathbf{x}_i) = \frac{\sum_{j=1}^{n} p_{u,j} x_{i,j}}{\sqrt{\sum_{j=1}^{n} p_{u,j}^2} \times \sqrt{\sum_{j=1}^{n} x_{i,j}^2}}
$$

#### 4. 평점 예측
사용자 $ u $ 가 아이템 $ i $ 를 선호할 가능성을 예측

$$
\hat{r}_{u, i} = \text{sim}(\mathbf{p}_u, \mathbf{x}_i)
$$

또는, 평가된 아이템의 가중 평균을 사용하여 예측할 수도 있음

$$
\hat{r}_{u, i} = \frac{\sum_{j \in I_u} \text{sim}(\mathbf{x}_j, \mathbf{x}_i) \cdot r_{u,j}}{\sum_{j \in I_u} \text{sim}(\mathbf{x}_j, \mathbf{x}_i)}
$$

여기서
- $ r_{u,j} $ : 사용자 $ u $ 가 아이템 $ j $ 에 부여한 평점
- $ \text{sim}(\mathbf{x}_j, \mathbf{x}_i) $ : 아이템 $ i $ 와 $ j $ 간의 유사도

#### 5. 추천
사용자에게 추천할 아이템 집합

$$
\hat{I}_u = \arg\max_{i \in I_{\text{all}}} \hat{r}_{u,i}
$$

여기서 $ I_{\text{all}} $ 은 전체 아이템 집합


---



## 1. Load Libraries

In [34]:
import pandas as pd
import numpy as np
import gensim
import requests
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec

## 2. Load Data

In [37]:
# load data
filename = "data/tmdb_5000_movies.csv"
movies = pd.read_csv(filename)

movies = movies[['id', 'title', 'overview', 'genres', 'keywords']]
movies.head()

,id,title,overview,genres,keywords
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."


In [38]:
# pre-processing
movies.fillna('', inplace=True)

# 장르와 키워드를 쉼표로 분리된 문자열로 변환
movies['genres'] = movies['genres'].apply(lambda x: ' '.join(x.replace("[", "").replace("]", "").replace("'", "").split(", ")))
movies['keywords'] = movies['keywords'].apply(lambda x: ' '.join(x.replace("[", "").replace("]", "").replace("'", "").split(", ")))

# 장르, 키워드, 줄거리를 합쳐서 하나의 특징 문자열 생성
# **단순 concat이 아닌, 각각 임베딩 후 가중치 곱하여 더하는 등 피쳐 임베딩에는 다양한 방법이 있음**
movies['features'] = movies['genres'] + " " + movies['keywords'] + " " + movies['overview']

# 각 영화의 특징을 단어 리스트로 변환 for Word2Vec
corpus = movies['features'].apply(lambda x: x.split()).tolist()
corpus[0][:20]

['{"id":',
 '28',
 '"name":',
 '"Action"}',
 '{"id":',
 '12',
 '"name":',
 '"Adventure"}',
 '{"id":',
 '14',
 '"name":',
 '"Fantasy"}',
 '{"id":',
 '878',
 '"name":',
 '"Science',
 'Fiction"}',
 '{"id":',
 '1463',
 '"name":']

## 3. Embedding Model

In [49]:
# Word2Vec 모델 학습
# pretrained도 있음. 이 모델을 가져와서 추가 학습(Fine-tuniing) 하는 것도 방법
w2v_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=2, workers=4)

# 특정 단어와 유사한 단어 확인
print(w2v_model.wv.most_similar('action'))

[('law', 0.9973795413970947), ('CIA', 0.997283399105072), ('executive', 0.997201144695282), ('heroic', 0.9968848824501038), ('class', 0.9963340163230896), ('political', 0.9962999224662781), ('setting', 0.9962477087974548), ('student', 0.9962361454963684), ('Jack', 0.9961742758750916), ('John', 0.9960528612136841)]


## 4. Item to Vector

In [51]:
def get_movie_vector(movie):
    words = movies.loc[movies['title'] == movie, 'features'].values[0].split()
    words = [w for w in words if w in w2v_model.wv]
    
    if len(words) == 0:
        return np.zeros(w2v_model.vector_size)
    
    return np.mean([w2v_model.wv[w] for w in words], axis=0)  # 단어 벡터들의 평균

# 모든 영화 벡터 생성
movie_vectors = np.array([get_movie_vector(title) for title in movies['title']], dtype=object)

# 기존 데이터프레임에 벡터 추가
movies['vector'] = list(movie_vectors)
movies.head()

,id,title,overview,genres,keywords,features,vector
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","{""id"": 28 ""name"": ""Action""} {""id"": 12 ""name"": ...","{""id"": 1463 ""name"": ""culture clash""} {""id"": 29...","{""id"": 28 ""name"": ""Action""} {""id"": 12 ""name"": ...","[-0.9081122875213623, 1.0658096075057983, -0.5..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","{""id"": 12 ""name"": ""Adventure""} {""id"": 14 ""name...","{""id"": 270 ""name"": ""ocean""} {""id"": 726 ""name"":...","{""id"": 12 ""name"": ""Adventure""} {""id"": 14 ""name...","[-0.8572640419006348, 0.9965201616287231, -0.3..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"{""id"": 28 ""name"": ""Action""} {""id"": 12 ""name"": ...","{""id"": 470 ""name"": ""spy""} {""id"": 818 ""name"": ""...","{""id"": 28 ""name"": ""Action""} {""id"": 12 ""name"": ...","[-0.7425779104232788, 1.0348178148269653, -0.4..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"{""id"": 28 ""name"": ""Action""} {""id"": 80 ""name"": ...","{""id"": 849 ""name"": ""dc comics""} {""id"": 853 ""na...","{""id"": 28 ""name"": ""Action""} {""id"": 80 ""name"": ...","[-0.7791153788566589, 0.9623020887374878, -0.3..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","{""id"": 28 ""name"": ""Action""} {""id"": 12 ""name"": ...","{""id"": 818 ""name"": ""based on novel""} {""id"": 83...","{""id"": 28 ""name"": ""Action""} {""id"": 12 ""name"": ...","[-0.7894370555877686, 1.0548986196517944, -0.3..."


## 5. User Profile Vector

In [56]:
def get_user_profile(user_movies):  # 평점 반영하지 않고 seen or not으로만 간단하게
    vectors = [get_movie_vector(movie) for movie in user_movies if movie in movies['title'].values]
    if len(vectors) == 0:
        return np.zeros(w2v_model.vector_size)
    return np.mean(vectors, axis=0)  # 사용자가 선호하는 영화 벡터들의 평균

# 사용자가 좋아하는 영화 입력
user_liked_movies = ["Inception", "Interstellar", "The Matrix"]
user_vector = get_user_profile(user_liked_movies)
user_vector

array([-0.8630559 ,  1.0825125 , -0.38381934,  0.17185014, -0.03859385,
       -0.4649216 ,  0.5010359 ,  0.8779258 , -1.2298661 , -0.39104542,
       -0.14614516, -1.2961942 ,  0.8578103 ,  1.323586  ,  0.12265492,
       -0.508803  ,  1.7758557 , -0.86016387, -0.7745681 , -1.2171159 ,
        1.5335423 , -0.04659058,  1.4546477 , -0.18494497,  0.5609943 ,
        0.2867738 , -0.38961807, -0.2017308 , -1.1654058 , -0.10293289,
       -0.50758904, -0.03662036,  0.9723582 , -1.4218575 , -0.04262307,
        1.0087565 ,  0.92747146, -0.21434276,  0.13343705,  0.07380779,
       -1.2019585 ,  0.6512812 ,  0.23517686, -0.5232896 , -0.23453695,
        0.33142892, -0.34740868, -0.61136115,  1.0833288 , -0.01716529,
       -0.33610216,  0.9403875 , -0.62749636,  0.9255521 ,  0.70905274,
       -1.4785389 ,  1.782391  , -0.5043789 , -0.59350944,  1.863509  ,
        0.5082372 , -0.44631943, -1.0640551 ,  0.11235722, -0.42408285,
        0.0765017 , -0.92969066, -1.1987243 , -0.82276744,  1.03

## 6. Recommendation

In [60]:
def recommend_movies(user_vector, top_n=10):
    # 사용자 벡터와 모든 영화 벡터 간 유사도 계산
    sim_scores = cosine_similarity([user_vector], movie_vectors).flatten()

    # 상위 N개 추천
    top_indices = np.argsort(sim_scores)[-top_n:][::-1]

    # 영화 제목, 장르, 유사도 출력
    return pd.DataFrame({
        "title": movies['title'].iloc[top_indices].values,
        "genres": movies['genres'].iloc[top_indices].values,
        "similarity": sim_scores[top_indices]
    })

print(user_liked_movies)
recommend_movies(user_vector)


['Inception', 'Interstellar', 'The Matrix']


,title,genres,similarity
0,Austin Powers: The Spy Who Shagged Me,"{""id"": 12 ""name"": ""Adventure""} {""id"": 35 ""name...",0.999701
1,The Mist,"{""id"": 878 ""name"": ""Science Fiction""} {""id"": 2...",0.999679
2,The Day After Tomorrow,"{""id"": 28 ""name"": ""Action""} {""id"": 12 ""name"": ...",0.999668
3,The Charge of the Light Brigade,"{""id"": 28 ""name"": ""Action""} {""id"": 12 ""name"": ...",0.999661
4,Guardians of the Galaxy,"{""id"": 28 ""name"": ""Action""} {""id"": 878 ""name"":...",0.999659
5,Mission: Impossible II,"{""id"": 12 ""name"": ""Adventure""} {""id"": 28 ""name...",0.999593
6,Dante's Peak,"{""id"": 28 ""name"": ""Action""} {""id"": 12 ""name"": ...",0.999580
7,The Dark Knight Rises,"{""id"": 28 ""name"": ""Action""} {""id"": 80 ""name"": ...",0.999570
8,On Her Majesty's Secret Service,"{""id"": 12 ""name"": ""Adventure""} {""id"": 28 ""name...",0.999549
9,End of Days,"{""id"": 28 ""name"": ""Action""} {""id"": 14 ""name"": ...",0.999546
